In [1]:
# Cell 1 — Load and label TRAIN columns (FD001)
# This reads the CMAPSS train file, assigns human-readable column names,
# and configures pandas to display all columns in previews.

from pathlib import Path
import pandas as pd

# Show all columns when printing/previewing dataframes
pd.set_option("display.max_columns", None)
# Let pandas use the available width (prevents truncated outputs)
pd.set_option("display.width", None)

# CMAPSS files are whitespace-delimited with no header row
train_path = Path("train_FD001.txt")
df_train = pd.read_csv(
    train_path,
    sep=r"\s+",          # split on one-or-more spaces
    header=None,          # no header in the raw file
    engine="python",     # more tolerant for whitespace parsing
)

# Defensive: sometimes trailing spaces create an extra empty column
# (all values are NaN). We drop it if present.
df_train = df_train.dropna(axis=1, how="all")

# Standard CMAPSS column layout:
# 2 ids (unit, cycle) + 3 operational settings + 21 sensor readings = 26 columns
col_names = [
    "unit",        # engine id
    "cycle",       # time step
    "setting_1",   # operating condition 1
    "setting_2",   # operating condition 2
    "setting_3",   # operating condition 3
    *[f"s{i}" for i in range(1, 22)],  # sensors s1..s21
]

# Fail fast if the file shape isn't what we expect (catches parsing issues)
if df_train.shape[1] != len(col_names):
    raise ValueError(
        f"Unexpected #columns in train: got {df_train.shape[1]}, expected {len(col_names)}. "
        "This usually means the file was parsed incorrectly."
    )

# Assign column names
df_train.columns = col_names

print("Train shape:", df_train.shape)
# Quick preview
df_train.head()

Train shape: (20631, 26)


,unit,cycle,setting_1,setting_2,setting_3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [2]:
# Cell 2 — Load and label TEST columns (FD001)
# The test file has the same columns as train, but each engine is truncated
# before failure. We'll label columns the same way for a consistent pipeline.

from pathlib import Path

test_path = Path("test_FD001.txt")
df_test = pd.read_csv(
    test_path,
    sep=r"\s+",      # whitespace-delimited
    header=None,      # no header row
    engine="python", # robust whitespace parsing
)

# Defensive: drop any all-NaN column created by trailing spaces
df_test = df_test.dropna(axis=1, how="all")

# Validate shape before assigning column names
if df_test.shape[1] != len(col_names):
    raise ValueError(
        f"Unexpected #columns in test: got {df_test.shape[1]}, expected {len(col_names)}."
    )

# Assign column names to match train
df_test.columns = col_names

print("Test shape:", df_test.shape)
df_test.head()

Test shape: (13096, 26)


,unit,cycle,setting_1,setting_2,setting_3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,2388.04,9050.17,1.3,47.20,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,2388.01,9054.42,1.3,47.50,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,2388.05,9056.96,1.3,47.50,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,554.07,2388.03,9045.29,1.3,47.28,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,554.16,2388.01,9044.55,1.3,47.31,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [3]:
# Cell 3 — Load RUL file for FD001
# RUL_FD001.txt contains ONE number per test engine: the RUL at the *final observed cycle*
# of that engine in the test set.

from pathlib import Path

rul_path = Path("RUL_FD001.txt")
df_rul = pd.read_csv(
    rul_path,
    sep=r"\s+",      # whitespace-delimited
    header=None,      # no header
    engine="python",
)

# Defensive: drop any empty column if present
df_rul = df_rul.dropna(axis=1, how="all")

# Name the only column
if df_rul.shape[1] != 1:
    raise ValueError(f"Unexpected RUL file shape: {df_rul.shape}. Expected 1 column.")

df_rul.columns = ["RUL"]
print("RUL shape:", df_rul.shape)
df_rul.head()

RUL shape: (100, 1)


,RUL
0,112
1,98
2,69
3,82
4,91


In [4]:
# Cell 4 — Create per-row RUL labels + choose feature columns
# Goal:
# 1) Create df_train_labeled with per-cycle RUL (train runs to failure).
# 2) Create df_test_labeled with per-cycle RUL by using RUL_FD001 (RUL at end-of-test)
#    and propagating it backward to earlier cycles.
# 3) Define a feature column list and drop near-constant columns on TRAIN only.

import numpy as np

# Ensure unit and cycle are integers (helps merges/groupby and avoids float surprises)
for _df in (df_train, df_test):
    _df["unit"] = _df["unit"].astype(int)
    _df["cycle"] = _df["cycle"].astype(int)

# Work on a copy so we don't mutate df_rul unexpectedly
# (useful if you re-run cells out of order)
df_rul = df_rul.copy()

# --- Align RUL rows to actual test unit IDs ---
# RUL_FD001.txt is typically ordered by test engine id (1..N), but we avoid assuming that.
# Instead, we map each RUL row to the sorted unique test units.
test_units_sorted = np.sort(df_test["unit"].unique())

# If these don't match, you likely mixed files (e.g., FD001 test with FD002 RUL)
if len(df_rul) != len(test_units_sorted):
    raise ValueError(
        f"RUL length ({len(df_rul)}) does not match #test units ({len(test_units_sorted)}). "
        "Make sure train/test/RUL are all FD001 and from the same CMAPSS release."
    )

# Add a 'unit' column to df_rul to support merges
# After this, df_rul has columns: [RUL, unit]
df_rul["unit"] = test_units_sorted

# --- TRAIN labels: RUL to failure ---
# For train, we observe each engine until failure.
# RUL at cycle t is: max_cycle(unit) - t
train_max_cycle = df_train.groupby("unit")["cycle"].max().rename("max_cycle")

df_train_labeled = df_train.merge(train_max_cycle, on="unit", how="left")
df_train_labeled["RUL"] = df_train_labeled["max_cycle"] - df_train_labeled["cycle"]
df_train_labeled = df_train_labeled.drop(columns=["max_cycle"])

# --- TEST labels: propagate RUL backwards ---
# In test, we do NOT observe failure. We only know RUL at the engine's last observed cycle.
# Let end_cycle(unit) be the last cycle in the test file for that unit.
# Then for any earlier cycle t:
#   RUL(unit, t) = RUL_end(unit) + (end_cycle(unit) - t)

test_end_cycle = df_test.groupby("unit")["cycle"].max().rename("end_cycle")

df_test_labeled = df_test.merge(test_end_cycle, on="unit", how="left")
df_test_labeled = df_test_labeled.merge(df_rul, on="unit", how="left")

# If any unit didn't get an RUL, merging/alignment failed
missing_rul_units = df_test_labeled.loc[df_test_labeled["RUL"].isna(), "unit"].unique()
if len(missing_rul_units) > 0:
    raise ValueError(f"Missing RUL values for units: {missing_rul_units[:10]} ...")

# Propagate RUL to every cycle row in the test set
df_test_labeled["RUL"] = df_test_labeled["RUL"] + (
    df_test_labeled["end_cycle"] - df_test_labeled["cycle"]
)

# Remove helper column
df_test_labeled = df_test_labeled.drop(columns=["end_cycle"])

# --- Quick sanity prints ---
print(
    "Train labeled shape:",
    df_train_labeled.shape,
    "RUL min/max:",
    (df_train_labeled["RUL"].min(), df_train_labeled["RUL"].max()),
)
print(
    "Test labeled shape:",
    df_test_labeled.shape,
    "RUL min/max:",
    (df_test_labeled["RUL"].min(), df_test_labeled["RUL"].max()),
)
print(
    "# train units:",
    df_train_labeled["unit"].nunique(),
    "# test units:",
    df_test_labeled["unit"].nunique(),
    "# RUL rows:",
    df_rul.shape[0],
)

# --- Define feature columns ---
# Inputs to the model: settings + sensor measurements
settings_cols = ["setting_1", "setting_2", "setting_3"]
sensor_cols = [f"s{i}" for i in range(1, 22)]
feature_cols_all = settings_cols + sensor_cols

# --- Drop near-constant features (TRAIN only) ---
# Columns with ~0 variance don't help learning and can cause numerical issues.
stds = df_train_labeled[feature_cols_all].std(numeric_only=True)
constant_cols = stds[stds < 1e-8].index.tolist()
feature_cols = [c for c in feature_cols_all if c not in constant_cols]

print("Constant/near-constant columns dropped:", constant_cols)
print("Final feature count:", len(feature_cols))

# Preview a few labeled rows with selected features
(df_train_labeled[["unit", "cycle", "RUL"] + feature_cols].head())

Train labeled shape: (20631, 27) RUL min/max: (np.int64(0), np.int64(361))
Test labeled shape: (13096, 27) RUL min/max: (np.int64(7), np.int64(340))
# train units: 100 # test units: 100 # RUL rows: 100
Constant/near-constant columns dropped: ['setting_3', 's1', 's5', 's10', 's16', 's18', 's19']
Final feature count: 17


,unit,cycle,RUL,setting_1,setting_2,s2,s3,s4,s6,s7,s8,s9,s11,s12,s13,s14,s15,s17,s20,s21
0,1,1,191,-0.0007,-0.0004,641.82,1589.70,1400.60,21.61,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190
1,1,2,190,0.0019,-0.0003,642.15,1591.82,1403.14,21.61,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236
2,1,3,189,-0.0043,0.0003,642.35,1587.99,1404.20,21.61,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442
3,1,4,188,0.0007,0.0000,642.35,1582.79,1401.87,21.61,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739
4,1,5,187,-0.0019,-0.0002,642.37,1582.85,1406.22,21.61,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044


In [5]:
# Cell 5 — Train/validation split by engine + windowing helper
# Why split by unit?
# If we split randomly by rows, the same engine appears in both train and validation,
# leaking information across cycles. Group split ensures validation engines are unseen.

import numpy as np
from sklearn.model_selection import GroupShuffleSplit

# --- Split engines (units) into train and validation sets ---
unit_split = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(
    unit_split.split(df_train_labeled, groups=df_train_labeled["unit"])
)

train_units = set(df_train_labeled.iloc[train_idx]["unit"].unique())
val_units = set(df_train_labeled.iloc[val_idx]["unit"].unique())

print(
    "Split units: train=",
    len(train_units),
    "val=",
    len(val_units),
    "overlap=",
    len(train_units & val_units),
)

# --- Windowing for sequence models (RNN/GRU/LSTM) ---
# We convert each engine's time series into overlapping windows.
# Each window is a tensor of shape (window_length, num_features).
# The target y is the RUL at the END of the window.

def make_windows(df, feature_cols, target_col="RUL", window=30, stride=1):
    X_list, y_list, unit_list, cycle_list = [], [], [], []

    # Process each engine independently
    for unit, g in df.groupby("unit"):
        g = g.sort_values("cycle")

        # Feature matrix and target vector for this engine
        Xv = g[feature_cols].to_numpy(dtype=np.float32)
        yv = g[target_col].to_numpy(dtype=np.float32)
        cv = g["cycle"].to_numpy(dtype=np.int32)

        # Skip engines shorter than the chosen window length
        if len(g) < window:
            continue

        # Slide a window over time
        for end in range(window - 1, len(g), stride):
            start = end - window + 1
            X_list.append(Xv[start : end + 1])  # (window, num_features)
            y_list.append(yv[end])              # scalar RUL at window end
            unit_list.append(unit)              # engine id
            cycle_list.append(cv[end])          # cycle at window end

    return (
        np.stack(X_list),
        np.array(y_list),
        np.array(unit_list),
        np.array(cycle_list),
    )

# --- Quick smoke test: build windows from a few units ---
# (This just verifies shapes; it is not training a model yet.)
sample_units = list(train_units)[:5]
sample_df = df_train_labeled[df_train_labeled["unit"].isin(sample_units)]

X_sample, y_sample, u_sample, c_sample = make_windows(
    sample_df,
    feature_cols,
    window=30,
    stride=5,
)

print("Window sample shapes:", X_sample.shape, y_sample.shape)

Split units: train= 80 val= 20 overlap= 0
Window sample shapes: (192, 30, 17) (192,)


In [21]:
# Cell 6 — Hierarchical Bayesian baseline (PyMC)
# Goal: a *hierarchical* model that shares information across engines (units),
# while allowing each engine to deviate from the global mean.
#
# Model (random-intercept hierarchical regression):
#   y_{u,t} ~ Normal(mu_{u,t}, sigma)
#   mu_{u,t} = alpha + b_u + X_{u,t} beta
#   b_u ~ Normal(0, tau)
#
# For VALIDATION we hold out entire engines (new units).
# For a *new* engine u*, the random effect b_{u*} is unknown;
# the predictive variance includes tau^2 (between-engine variability).

import time
import numpy as np
import pandas as pd

import pymc as pm
import arviz as az

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- Fast vs full settings ---
# If your kernel is taking a long time (tens of minutes), keep FAST_DEBUG=True.
# Once it runs end-to-end reliably, switch FAST_DEBUG=False and scale up gradually.
FAST_DEBUG = True

random_seed = 42

if FAST_DEBUG:
    cycle_stride = 25       # more aggressive downsampling
    max_train_rows = 4000
    max_val_rows = 1500
    advi_steps = 4000
    posterior_draws = 1000  # used for ADVI posterior and for predictive draws
else:
    cycle_stride = 5
    max_train_rows = 20000
    max_val_rows = 8000
    advi_steps = 20000
    posterior_draws = 1000

# Prefer NUTS for better uncertainty calibration on the smaller FAST_DEBUG dataset.
# For larger runs, fall back to ADVI for speed.
INFERENCE = "nuts" if FAST_DEBUG else "advi"

# NUTS presets (set NUTS_PRESET="final" before your final report run)
NUTS_PRESET = "fast" if FAST_DEBUG else "final"  # "fast" | "final"
if NUTS_PRESET == "fast":
    nuts_tune = 300
    nuts_draws = 300
    nuts_chains = 2
    nuts_target_accept = 0.9
    nuts_checks = False
else:
    nuts_tune = 750
    nuts_draws = 750
    nuts_chains = 2
    nuts_target_accept = 0.9
    nuts_checks = True

t0 = time.perf_counter()

# Make sure prerequisites exist (run Cells 1–5 first)
required_vars = [
    "df_train_labeled",
    "feature_cols",
    "train_units",
    "val_units",
]
missing = [v for v in required_vars if v not in globals()]
if missing:
    raise RuntimeError(f"Missing variables: {missing}. Run Cells 1–5 first.")

# --- Build a tabular dataset (one row = one cycle) ---
# We downsample within each engine to keep inference fast.
def downsample_by_unit(df, stride):
    df_sorted = df.sort_values(["unit", "cycle"]).reset_index(drop=True)
    keep = (df_sorted.groupby("unit").cumcount() % stride) == 0
    return df_sorted.loc[keep].reset_index(drop=True)

train_tab = df_train_labeled[df_train_labeled["unit"].isin(train_units)].copy()
val_tab = df_train_labeled[df_train_labeled["unit"].isin(val_units)].copy()

train_tab = downsample_by_unit(train_tab, cycle_stride)
val_tab = downsample_by_unit(val_tab, cycle_stride)

# Optional hard caps (extra safety if stride is small)
if len(train_tab) > max_train_rows:
    train_tab = train_tab.sample(n=max_train_rows, random_state=random_seed)
if len(val_tab) > max_val_rows:
    val_tab = val_tab.sample(n=max_val_rows, random_state=random_seed)

print("[prep] train rows:", len(train_tab), "val rows:", len(val_tab), "stride:", cycle_stride)
print("[prep] elapsed sec:", round(time.perf_counter() - t0, 2))

# --- Features/target ---
X_train_raw = train_tab[feature_cols].to_numpy(dtype=float)
y_train = train_tab["RUL"].to_numpy(dtype=float)
unit_train = train_tab["unit"].to_numpy(dtype=int)

X_val_raw = val_tab[feature_cols].to_numpy(dtype=float)
y_val = val_tab["RUL"].to_numpy(dtype=float)
unit_val = val_tab["unit"].to_numpy(dtype=int)

# Standardize features using TRAIN stats only (avoid leakage)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw).astype(np.float32)
X_val = scaler.transform(X_val_raw).astype(np.float32)

# Standardize target for stable inference; back-transform predictions for metrics
y_mean = float(np.mean(y_train))
y_std = float(np.std(y_train) + 1e-8)
y_train_s = ((y_train - y_mean) / y_std).astype(np.float32)
y_val_s = ((y_val - y_mean) / y_std).astype(np.float32)

print(
    "[prep] y_train (raw) mean:", round(y_mean, 3),
    "std:", round(y_std, 3),
    "min:", round(float(np.min(y_train)), 3),
    "max:", round(float(np.max(y_train)), 3),
)

# Map training units to 0..(U-1) for indexing b_u
train_units_sorted = np.sort(np.unique(unit_train))
unit_to_idx = {u: i for i, u in enumerate(train_units_sorted)}
unit_idx_train = np.array([unit_to_idx[u] for u in unit_train], dtype=int)

print("[prep] # train units:", len(train_units_sorted), "# features:", X_train.shape[1])
print("[prep] inference:", INFERENCE, "preset:", NUTS_PRESET)
print("[prep] elapsed sec:", round(time.perf_counter() - t0, 2))

coords = {
    "unit": train_units_sorted,
    "feature": np.array(feature_cols),
}

with pm.Model(coords=coords) as hier_model:
    # Data containers
    X = pm.Data("X", X_train, dims=("obs", "feature"))
    unit_idx = pm.Data("unit_idx", unit_idx_train, dims="obs")
    y = pm.Data("y", y_train_s, dims="obs")

    # Priors (in standardized y-space)
    alpha = pm.Normal("alpha", mu=0.0, sigma=1.0)

    # Hierarchical (random effects): per-engine intercept b_u
    # Non-centered parameterization helps NUTS and often reduces pathologies.
    tau = pm.HalfNormal("tau", sigma=1.0)
    b_unit_offset = pm.Normal("b_unit_offset", mu=0.0, sigma=1.0, dims="unit")
    b_unit = pm.Deterministic("b_unit", b_unit_offset * tau, dims="unit")

    # Global feature effects
    beta = pm.Normal("beta", mu=0.0, sigma=1.0, dims="feature")

    # Observation noise
    sigma = pm.HalfNormal("sigma", sigma=1.0)

    # Expected value
    mu = alpha + b_unit[unit_idx] + pm.math.dot(X, beta)

    # Likelihood
    pm.Normal("RUL_obs", mu=mu, sigma=sigma, observed=y, dims="obs")

    if INFERENCE == "nuts":
        print("[fit] starting NUTS: tune=", nuts_tune, "draws=", nuts_draws, "chains=", nuts_chains)
        print("[fit] elapsed sec:", round(time.perf_counter() - t0, 2))
        posterior = pm.sample(
            draws=nuts_draws,
            tune=nuts_tune,
            chains=nuts_chains,
            cores=1,
            target_accept=nuts_target_accept,
            random_seed=random_seed,
            progressbar=True,
            compute_convergence_checks=nuts_checks,
        )
    else:
        print("[fit] starting ADVI steps:", advi_steps)
        print("[fit] elapsed sec:", round(time.perf_counter() - t0, 2))
        approx = pm.fit(
            n=advi_steps,
            method="advi",
            random_seed=random_seed,
            progressbar=True,
        )
        posterior = approx.sample(posterior_draws, random_seed=random_seed)

print("[fit] done. elapsed sec:", round(time.perf_counter() - t0, 2))
if INFERENCE == "nuts" and nuts_checks:
    print("[fit] quick convergence summary (key scalars)")
    display(az.summary(posterior, var_names=["alpha", "tau", "sigma"]))

# --- Predict on held-out engines (new units) ---
# For unseen units, E[b_u] = 0, and Var(b_u) = tau^2.
# Therefore predictive sd becomes sqrt(sigma^2 + tau^2).

a = posterior.posterior["alpha"].values.reshape(-1)
tau_s = posterior.posterior["tau"].values.reshape(-1)
sigma_s = posterior.posterior["sigma"].values.reshape(-1)
beta_s = posterior.posterior["beta"].values.reshape(-1, X_val.shape[1])

# Compute mu for each draw (standardized y): mu = alpha + X beta
mu_draws_s = a[:, None] + beta_s @ X_val.T

# Predictive SD for new units includes both noise and between-engine uncertainty (standardized y)
pred_sd_s = np.sqrt(sigma_s[:, None] ** 2 + tau_s[:, None] ** 2)

rng = np.random.default_rng(random_seed)
y_pred_draws_s = rng.normal(loc=mu_draws_s, scale=pred_sd_s)

# Back-transform to raw RUL scale
y_pred_draws_unclipped = y_mean + y_std * y_pred_draws_s
y_pred_draws = np.clip(y_pred_draws_unclipped, 0.0, None)

# Posterior predictive summaries (raw scale)
pred_mean = y_pred_draws.mean(axis=0)
pred_lo = np.quantile(y_pred_draws, 0.05, axis=0)
pred_hi = np.quantile(y_pred_draws, 0.95, axis=0)

rmse = float(np.sqrt(mean_squared_error(y_val, pred_mean)))
mae = float(mean_absolute_error(y_val, pred_mean))
coverage_90 = float(np.mean((y_val >= pred_lo) & (y_val <= pred_hi)))

print(
    f"HierBayes ({INFERENCE}/{NUTS_PRESET}) | RMSE={rmse:.3f} MAE={mae:.3f} | 90% PI coverage={coverage_90:.3f}"
)
print(
    "[pred] mean(pred_mean)=", round(float(np.mean(pred_mean)), 3),
    "min=", round(float(np.min(pred_mean)), 3),
    "max=", round(float(np.max(pred_mean)), 3),
)
print(
    "[pred] % unclipped draws < 0:",
    round(float(np.mean(y_pred_draws_unclipped < 0)) * 100, 2),
)
print("[all] total elapsed sec:", round(time.perf_counter() - t0, 2))

results = pd.DataFrame(
    {
        "unit": unit_val,
        "cycle": val_tab["cycle"].to_numpy(dtype=int),
        "RUL_true": y_val,
        "RUL_pred_mean": pred_mean,
        "RUL_p05": pred_lo,
        "RUL_p95": pred_hi,
    }
)
results.head(10)

c:\Users\lalot\anaconda3\envs\bayes_ml\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
Initializing NUTS using jitter+adapt_diag...


[prep] train rows: 702 val rows: 173 stride: 25
[prep] elapsed sec: 0.01
[prep] y_train (raw) mean: 113.779 std: 72.492 min: 0.0 max: 361.0
[prep] # train units: 80 # features: 17
[prep] inference: nuts preset: fast
[prep] elapsed sec: 0.01
[fit] starting NUTS: tune= 300 draws= 300 chains= 2
[fit] elapsed sec: 0.03


Sequential sampling (2 chains in 1 job)
NUTS: [alpha, tau, b_unit_offset, beta, sigma]


Output()

c:\Users\lalot\anaconda3\envs\bayes_ml\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Sampling 2 chains for 300 tune and 300 draw iterations (600 + 600 draws total) took 1292 seconds.


[fit] done. elapsed sec: 1292.24
HierBayes (nuts/fast) | RMSE=42.027 MAE=31.713 | 90% PI coverage=0.948
[pred] mean(pred_mean)= 110.643 min= 1.811 max= 220.45
[pred] % unclipped draws < 0: 10.86
[all] total elapsed sec: 1292.24


,unit,cycle,RUL_true,RUL_pred_mean,RUL_p05,RUL_p95
0,1,1,191.0,152.526757,66.361283,235.606600
1,1,26,166.0,181.179990,98.405808,262.257943
2,1,51,141.0,161.853576,81.757571,245.759119
3,1,76,116.0,158.284785,81.860708,240.577943
4,1,101,91.0,141.704874,60.470968,227.967382
5,1,126,66.0,107.922758,24.899012,187.560911
6,1,151,41.0,76.362958,0.000000,151.177122
7,1,176,16.0,41.731411,0.000000,120.315134
8,5,1,268.0,184.645518,103.251641,256.487284
9,5,26,243.0,213.482451,132.785997,294.317510


In [22]:
# Cell 7 — Diagnostics for calibration & realism
# Assumes Cell 6 has produced: y_val, pred_mean, y_pred_draws, results

import numpy as np
import pandas as pd

if "y_pred_draws" not in globals() or "y_val" not in globals() or "pred_mean" not in globals():
    raise RuntimeError("Run Cell 6 first (needs y_pred_draws, y_val, pred_mean).")

y_val_arr = np.asarray(y_val)
pred_mean_arr = np.asarray(pred_mean)
draws = np.asarray(y_pred_draws)  # shape: (n_draws, n_obs)

# If available, report how many raw draws were negative BEFORE clipping to 0
if "y_pred_draws_unclipped" in globals():
    draws_unclipped = np.asarray(y_pred_draws_unclipped)
    neg_unclipped = float(np.mean(draws_unclipped < 0))
else:
    neg_unclipped = float(np.mean(draws < 0))

print("[diag] n_obs:", y_val_arr.size, "n_draws:", draws.shape[0])
print("[diag] % pred_mean < 0:", round(float(np.mean(pred_mean_arr < 0)) * 100, 2))
print("[diag] % predictive draws < 0 (after any clipping):", round(float(np.mean(draws < 0)) * 100, 2))
print("[diag] % predictive draws < 0 (unclipped):", round(neg_unclipped * 100, 2))

interval_specs = [
    (0.25, 0.75, 0.50),
    (0.10, 0.90, 0.80),
    (0.05, 0.95, 0.90),
]

rows = []
for qlo, qhi, nominal in interval_specs:
    lo = np.quantile(draws, qlo, axis=0)
    hi = np.quantile(draws, qhi, axis=0)
    coverage = float(np.mean((y_val_arr >= lo) & (y_val_arr <= hi)))
    width = hi - lo
    rows.append(
        {
            "interval": f"{int(qlo*100)}–{int(qhi*100)}%",
            "nominal": nominal,
            "coverage": coverage,
            "mean_width": float(np.mean(width)),
            "median_width": float(np.median(width)),
        }
    )

diag = pd.DataFrame(rows)
display(diag)

# Error summary (all validation rows)
abs_err = np.abs(pred_mean_arr - y_val_arr)
print("[diag] MAE:", round(float(np.mean(abs_err)), 3))
print("[diag] Median AE:", round(float(np.median(abs_err)), 3))
print("[diag] 90th pct AE:", round(float(np.quantile(abs_err, 0.90)), 3))

# Evaluation at each validation unit's LAST observed cycle *within the downsampled validation table*
if "results" in globals() and isinstance(results, pd.DataFrame) and len(results) > 0:
    last_idx = results.groupby("unit")["cycle"].idxmax()
    results_last = results.loc[last_idx].copy().sort_values(["unit"]).reset_index(drop=True)

    y_last = results_last["RUL_true"].to_numpy(dtype=float)
    p_last = results_last["RUL_pred_mean"].to_numpy(dtype=float)
    lo_last = results_last["RUL_p05"].to_numpy(dtype=float)
    hi_last = results_last["RUL_p95"].to_numpy(dtype=float)

    rmse_last = float(np.sqrt(np.mean((y_last - p_last) ** 2)))
    mae_last = float(np.mean(np.abs(y_last - p_last)))
    cov90_last = float(np.mean((y_last >= lo_last) & (y_last <= hi_last)))

    print("[last-cycle/downsampled] n_units:", results_last.shape[0])
    print(f"[last-cycle/downsampled] RMSE={rmse_last:.3f} MAE={mae_last:.3f} | 90% PI coverage={cov90_last:.3f}")
    display(results_last.head(10))

# Evaluation at each validation unit's TRUE last cycle (from full df_train_labeled)
needed = ["df_train_labeled", "val_units", "feature_cols", "scaler", "a", "beta_s", "tau_s", "sigma_s", "y_mean", "y_std", "random_seed"]
missing_needed = [k for k in needed if k not in globals()]
if len(missing_needed) == 0:
    val_full = df_train_labeled[df_train_labeled["unit"].isin(val_units)].copy()
    val_last_trueend = (
        val_full.sort_values(["unit", "cycle"])
        .groupby("unit", as_index=False)
        .tail(1)
        .reset_index(drop=True)
    )

    X_last_raw = val_last_trueend[feature_cols].to_numpy(dtype=float)
    X_last = scaler.transform(X_last_raw).astype(np.float32)
    y_last_trueend = val_last_trueend["RUL"].to_numpy(dtype=float)
    unit_last_trueend = val_last_trueend["unit"].to_numpy(dtype=int)
    cycle_last_trueend = val_last_trueend["cycle"].to_numpy(dtype=int)

    # Posterior predictive for new units at these rows
    mu_draws_s_last = a[:, None] + beta_s @ X_last.T
    pred_sd_s_last = np.sqrt(sigma_s[:, None] ** 2 + tau_s[:, None] ** 2)
    rng_last = np.random.default_rng(int(random_seed) + 123)
    y_pred_draws_s_last = rng_last.normal(loc=mu_draws_s_last, scale=pred_sd_s_last)
    y_pred_draws_last_unclipped = y_mean + y_std * y_pred_draws_s_last
    y_pred_draws_last = np.clip(y_pred_draws_last_unclipped, 0.0, None)

    pred_mean_last_trueend = y_pred_draws_last.mean(axis=0)
    pred_lo_last_trueend = np.quantile(y_pred_draws_last, 0.05, axis=0)
    pred_hi_last_trueend = np.quantile(y_pred_draws_last, 0.95, axis=0)

    rmse_last_trueend = float(np.sqrt(np.mean((y_last_trueend - pred_mean_last_trueend) ** 2)))
    mae_last_trueend = float(np.mean(np.abs(y_last_trueend - pred_mean_last_trueend)))
    cov90_last_trueend = float(
        np.mean((y_last_trueend >= pred_lo_last_trueend) & (y_last_trueend <= pred_hi_last_trueend))
    )

    print("[last-cycle/trueend] n_units:", y_last_trueend.size)
    print(
        f"[last-cycle/trueend] RMSE={rmse_last_trueend:.3f} MAE={mae_last_trueend:.3f} "
        f"| 90% PI coverage={cov90_last_trueend:.3f}"
    )
    print(
        "[last-cycle/trueend] % unclipped draws < 0:",
        round(float(np.mean(y_pred_draws_last_unclipped < 0)) * 100, 2),
    )

    results_last_trueend = pd.DataFrame(
        {
            "unit": unit_last_trueend,
            "cycle": cycle_last_trueend,
            "RUL_true": y_last_trueend,
            "RUL_pred_mean": pred_mean_last_trueend,
            "RUL_p05": pred_lo_last_trueend,
            "RUL_p95": pred_hi_last_trueend,
        }
    )
    display(results_last_trueend.head(10))
else:
    print("[last-cycle/trueend] skipped; missing:", missing_needed)

# Worst cases (helps see where the model fails)
if "results" in globals() and isinstance(results, pd.DataFrame):
    worst = results.copy()
    worst["abs_err"] = abs_err
    display(worst.sort_values("abs_err", ascending=False).head(10))

[diag] n_obs: 173 n_draws: 600
[diag] % pred_mean < 0: 0.0
[diag] % predictive draws < 0 (after any clipping): 0.0
[diag] % predictive draws < 0 (unclipped): 10.86


,interval,nominal,coverage,mean_width,median_width
0,25–75%,0.5,0.549133,59.588722,64.805774
1,10–90%,0.8,0.843931,113.182558,122.854330
2,5–95%,0.9,0.947977,143.677100,157.363193


[diag] MAE: 31.713
[diag] Median AE: 26.834
[diag] 90th pct AE: 70.417
[last-cycle/downsampled] n_units: 20
[last-cycle/downsampled] RMSE=9.095 MAE=6.282 | 90% PI coverage=1.000


,unit,cycle,RUL_true,RUL_pred_mean,RUL_p05,RUL_p95
0,1,176,16.0,41.731411,0.0,120.315134
1,5,251,18.0,15.144089,0.0,69.337077
2,11,226,14.0,13.911727,0.0,68.184195
3,13,151,12.0,7.679619,0.0,47.071606
4,19,151,7.0,10.919906,0.0,55.234127
5,23,151,17.0,22.615239,0.0,89.544320
6,31,226,8.0,16.109089,0.0,76.564227
7,32,176,15.0,13.441193,0.0,65.340174
8,34,176,19.0,40.107779,0.0,113.202961
9,40,176,12.0,6.052041,0.0,39.215576


[last-cycle/trueend] n_units: 20
[last-cycle/trueend] RMSE=2.967 MAE=2.332 | 90% PI coverage=1.000
[last-cycle/trueend] % unclipped draws < 0: 90.37


,unit,cycle,RUL_true,RUL_pred_mean,RUL_p05,RUL_p95
0,1,192,0.0,7.125704,0.0,47.350527
1,5,269,0.0,1.030936,0.0,0.000000
2,11,240,0.0,1.133585,0.0,1.971975
3,13,163,0.0,0.551438,0.0,0.000000
4,19,158,0.0,1.731265,0.0,14.394951
5,23,168,0.0,2.818602,0.0,17.638670
6,31,234,0.0,6.108953,0.0,39.084220
7,32,191,0.0,0.181948,0.0,0.000000
8,34,195,0.0,0.210527,0.0,0.000000
9,40,188,0.0,2.792888,0.0,23.356507


,unit,cycle,RUL_true,RUL_pred_mean,RUL_p05,RUL_p95,abs_err
156,84,1,266.0,95.786698,24.196856,172.541642,170.213302
157,84,26,241.0,118.416996,38.026107,198.621419,122.583004
76,40,1,187.0,71.353780,0.000000,145.869110,115.646220
158,84,51,216.0,110.040946,36.201120,182.680576,105.959054
146,81,1,239.0,143.725753,61.880233,221.574173,95.274247
147,81,26,214.0,120.884604,36.894789,202.348444,93.115396
159,84,76,191.0,100.791626,26.379841,179.112269,90.208374
87,45,76,82.0,166.737867,81.830041,257.130720,84.737867
8,5,1,268.0,184.645518,103.251641,256.487284,83.354482
61,32,26,165.0,82.274177,0.000000,164.088988,82.725823
